In [189]:
result_folder = "/home/yatin/hpchome/pratheek/PartialLabelLearning/results/birdsong"

In [190]:
import os,string
path = result_folder
path = os.path.normpath(path)
res = []
for root,dirs,files in os.walk(path, topdown=True):
    depth = root[len(path) + len(os.path.sep):].count(os.path.sep)
    if depth == 2:
        # We're currently two directories in, so all subdirs have depth 3
        res += [os.path.join(root, d) for d in dirs]
        dirs[:] = [] # Don't recurse any deeper

In [191]:
columns = ['dataset','technique','fold','real_test_acc','train_IOU','train_IOU_neg','val_IOU','val_IOU_neg','test_IOU','test_IOU_neg','surrogate_test_acc', 'surrogate_train_acc', 
'real_train_acc', 'surrogate_val_acc', 'real_val_acc',  'train_confidence', 'val_confidence', 'test_confidence','best_epoch']

In [ ]:
import pandas as pd
import time 
data = pd.DataFrame(columns=columns) 
count = 0
for folder in res:
    
    logfile = os.path.join(folder, "logs","log.json")
    if(not(os.path.exists(logfile))):
        continue     
    
    count+=1
    
    df = pd.read_json(logfile, lines=True)
    df = df[df["epoch"] == -1]
    
    names = folder.split("/")
    for index, row in df.iterrows():
        if(row["epoch"] == -1):
            data.loc[count] = row
            data.loc[count,"dataset"] = names[-4]
            #data.loc[count,"model"] = names[-3]
            data.loc[count,"technique"] = names[-2]
            data.loc[count,"fold"] = names[-1]

In [110]:
a = data[columns].groupby(['dataset','technique']).mean().reset_index()
b = a.set_index(['dataset','technique']).unstack(level=0)
b = b.reindex(sorted(b.columns), axis=1)

b.to_csv('results_xy.csv')

In [188]:
pandas_list = []

for dataset_name in ['BirdSong']:
    c = data[columns]

    c = c.loc[c['technique'].str.contains('pretrain') | c['technique'].str.contains('cc_loss') | c['technique'].str.contains('freeze') | c['technique'].str.contains('pq') ]
    c = c.loc[c['dataset'].str.endswith(dataset_name)]
    c = c.loc[~c['technique'].str.contains('_1_')]

    cc = c.loc[c['technique'].str.contains('cc_loss')]
    #cc = cc.loc[~cc['technique'].str.contains('1e-05')]
    #cc = cc.loc[~cc['technique'].str.contains('1_0.001_')]

    cc['tech'] = 'cc'
    cc = cc.loc[cc.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]


    lstm = c.loc[c['technique'].str.contains('weighted_loss_xy_lstm_iexplr')]
    #print(lstm)
    lstm = lstm.loc[~lstm['technique'].str.contains('fully')]

    lstm_pq = lstm.loc[lstm['technique'].str.endswith('_100_pq')]
    lstm_freeze = lstm.loc[lstm['technique'].str.contains('_100_freeze')]
    lstm_pretrain = lstm.loc[lstm['technique'].str.contains('_100_pretrain')]
    
    lstm_pq['tech'] = 'lstm'
    lstm_freeze['tech'] = 'lstm_freeze'
    lstm_pretrain['tech'] = 'lstm_freeze+pretrain'

    lstm_pq = lstm_pq.loc[lstm_pq.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]
    lstm_freeze = lstm_freeze.loc[lstm_freeze.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]
    lstm_pretrain = lstm_pretrain.loc[lstm_pretrain.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]

    c = pd.concat([cc, lstm_pq, lstm_freeze, lstm_pretrain])

    #c = c[['real_test_acc','surrogate_val_acc','real_val_acc']]
    c = c.reindex(sorted(c.columns), axis=1)
    #c = c[c['dataset'] == 'MSRCv2_lstm']
    #print(dataset_name)
    d = c[['train_IOU','train_IOU_neg','test_IOU','test_IOU_neg','real_test_acc']].mean().tolist()
    print(dataset_name,end=" ")
    for i in d:
        print("{:8.2f}".format(i),end=" ")
    print("\n")
    c = c.pivot(index="tech", columns=["fold"])
    #c = c.set_index(['dataset','tech','fold']).unstack(level=0)

    c.to_csv('results_Yahoo.csv')
    
    
    pandas_list.append(c)
res = pd.concat(pandas_list)
res.to_csv("A_result.csv")

BirdSong      nan      nan      nan      nan    67.25 



/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [20]:
import pandas as pd
data = pd.DataFrame(columns=columns) 
count = 0
for folder in res:
    
    logfile = os.path.join(folder, "logs","log.json_lstm")
    if(not(os.path.exists(logfile))):
        continue
    df = pd.read_json(logfile, lines=True)
    df = df[df["epoch"] == -2]
    #print(df.columns.values)
    names = folder.split("/")
    for index, row in df.iterrows():
        
        if(row["epoch"] == -2):
            data.loc[count] = row
            data.loc[count,"dataset"] = names[-4]
            #data.loc[count,"model"] = names[-3]
            data.loc[count,"technique"] = names[-2]
            data.loc[count,"fold"] = names[-1]
            count+=1


c = data[columns]
print(data)
c = c.loc[c['technique'].str.contains('pretrain') | c['technique'].str.contains('cc_loss') | c['technique'].str.contains('freeze') | c['technique'].str.contains('pq') ]
c = c.loc[c['dataset'].str.endswith('MSRCv2')]
c = c.loc[~c['technique'].str.contains('_1_')]

cc = c.loc[c['technique'].str.contains('cc_loss')]
cc = cc.loc[~cc['technique'].str.contains('1e-05')]
cc = cc.loc[~cc['technique'].str.contains('1_0.001_')]

cc['tech'] = 'cc'
cc = cc.loc[cc.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]


lstm = c.loc[c['technique'].str.contains('weighted_loss_xy_lstm_iexplr')]
#print(lstm)
lstm = lstm.loc[~lstm['technique'].str.contains('fully')]

lstm_pq = lstm.loc[lstm['technique'].str.endswith('_100_pq')]
lstm_freeze = lstm.loc[lstm['technique'].str.contains('_100_freeze')]
lstm_pretrain = lstm.loc[lstm['technique'].str.contains('_100_pretrain')]

lstm_pq['tech'] = 'lstm'
lstm_freeze['tech'] = 'lstm_freeze'
lstm_pretrain['tech'] = 'lstm_freeze+pretrain'

lstm_pq = lstm_pq.loc[lstm_pq.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]
lstm_freeze = lstm_freeze.loc[lstm_freeze.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]
lstm_pretrain = lstm_pretrain.loc[lstm_pretrain.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]

c = pd.concat([cc, lstm_pq, lstm_freeze, lstm_pretrain])

c = c.reindex(sorted(c.columns), axis=1)
#c = c[c['dataset'] == 'MSRCv2_lstm']
c = c.pivot(index="tech", columns=["dataset","fold"])
print(c.head())
#c = c.set_index(['dataset','tech','fold']).unstack(level=0)

c.to_csv('results_xy_lstm_val_comb.csv')

   dataset                                          technique fold  \
0   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    1   
1   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    3   
2   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    9   
3   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.001_0.0001...    4   
4   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    1   
..     ...                                                ...  ...   
80  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    0   
81  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    2   
82  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    4   
83  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    6   
84  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    8   

    real_test_acc  train_IOU  train_IOU_neg   val_IOU  val_IOU_neg  test_IOU  \
0       42.857143  -1.000000      -1.000000 -1.000000    -1.000000 -1.000000   

/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p